In [1]:
# Importing necessary packages
import plotly.express as px
from pandas_datareader import wb
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

In [2]:
# get FP.CPI.TOTL.ZG for the G7 countries from 1991 to 2021 from the World Bank
df1 = wb.download(indicator='FP.CPI.TOTL.ZG', country=['US', 'CA', 'GB', 'FR', 'DE', 'IT', 'JP'], start=2000, end=2021)

In [3]:
def sort_frame(df):
    # make the year the index
    df = df.reset_index().set_index('year')
    # make the columns the countries
    df = df.pivot(columns='country', values='FP.CPI.TOTL.ZG')
    # remove potential NaNs
    df = df.dropna()
    return df

In [4]:
# sort df
df1 = sort_frame(df1)

In [5]:
# add column with the average inflation rate
df1['Average'] = df1.mean(axis=1)

In [6]:
# my custom function to make the plot more readable
def ez_interact (fig):
    # show only every other year on the x-axis without referencing a specific dataframe
    fig.update_xaxes(tickvals=fig.data[0].x[::2])
    # increase ticks in y axis
    fig.update_yaxes(nticks=20)
    # adjust lines to make them more visible
    fig.update_traces(line=dict(width=2))
    # make average line dashed and thicker and black
    fig.update_traces(line=dict(width=3, dash='dash'), selector=dict(name='Average'))
    # make lines darker
    fig.update_traces(line=dict(color='black'), selector=dict(name='Average'))
    # add range slider
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=13, label="2008 Financial Crisis", step="year", stepmode="backward"),
                    dict(count=7, label="2014 Russo-Ukrainian War", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    return fig

In [7]:
# create a line plot with the G7 countries and the average
fig = px.line(df1, title='Inflation Rate in the G7 Countries and the Average')

# fix the layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Inflation Rate (%)',
    legend_title='Country'
)

# call my custom function
ez_interact(fig)

fig.show()

In [8]:
# new data frame with inflation data from Russia, China and India
df2 = wb.download(indicator='FP.CPI.TOTL.ZG', country=['RU', 'CN', 'IN'], start=2000, end=2021)
# sort df
df2 = sort_frame(df2)
# add column with the G7 average from df1
df2['Average'] = df1['Average']
# rename column
df2 = df2.rename(columns={'Average': 'G7 Average'})

In [9]:
# create an interactive line plot with the G7 countries and the average
fig2 = px.line(df2, title='Inflation Rate in Russia, China and India and the G7')

# fix the layout
fig2.update_layout(
    xaxis_title='Year',
    yaxis_title='Inflation Rate (%)',
    legend_title='Country'
)

# style the graph
ez_interact(fig2)

fig2.show()

In [10]:
# Store url to CSS file in a variable...
dbc_css = 'https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates@V1.0.2/dbc.min.css'

# ...and add the variable to the list of external stylesheets
app = JupyterDash(external_stylesheets = [dbc.themes.LUMEN, dbc_css])

description = """
Strategic overview of inflation in the G7 countries and Russia, China and India, from 2000 to 2021.
Buttons set the start date of the graph to the relevant year, and the range slider allows for zooming in.

"""

app.layout = dbc.Container(
    children = [
        # Header
        html.H1('Inflation in the G7 and Russia, China and India'),
        dcc.Markdown(description),
        
        dbc.Row([
            dbc.Col(dcc.Graph(figure = fig), width = 6),
            dbc.Col(dcc.Graph(figure = fig2), width = 6)
            ]
        )            
    ],
    className = 'dbc'   # link application with CSS file
)

app.run_server(mode = 'inline')